Install [Pyomo](https://www.pyomo.org/)

In [1]:
!pip install -q pyomo

Install CBC-solver [COIN-OR CBC solver](https://www.coin-or.org/)

In [2]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [3]:
import pyomo.environ as pyo

A = ['Syncreon1', 'Syncreon2', 'Ceva1', 'Ceva2']
b = {'Syncreon1':24, 'Syncreon2':50, 'Ceva1':18, 'Ceva2':20}
w = {'Syncreon1':100, 'Syncreon2':400, 'Ceva1':150, 'Ceva2':300}
budget_max = 170

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

model.obj = pyo.Objective(
    expr = sum( b[i]*model.x[i] for i in A ),
    sense = pyo.maximize )

model.budget_con = pyo.Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= budget_max )

#opt = pyo.SolverFactory('glpk')
opt = pyo.SolverFactory('cbc')
opt_success = opt.solve(model)

model.pprint()
# Print the values of the decision variables
print("\nSelected items:")
for i in A:
    print(f"{i}: {model.x[i].value}")

print(f"Objective: {model.obj()}")

1 Var Declarations
    x : Size=4, Index={Syncreon1, Syncreon2, Ceva1, Ceva2}
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
            Ceva1 :     0 :   0.0 :     1 : False : False : Binary
            Ceva2 :     0 :   0.0 :     1 : False : False : Binary
        Syncreon1 :     0 :   1.0 :     1 : False : False : Binary
        Syncreon2 :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 24*x[Syncreon1] + 50*x[Syncreon2] + 18*x[Ceva1] + 20*x[Ceva2]

1 Constraint Declarations
    budget_con : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                              : Upper : Active
        None :  -Inf : 100*x[Syncreon1] + 400*x[Syncreon2] + 150*x[Ceva1] + 300*x[Ceva2] : 170.0 :   True

3 Declarations: x obj budget_con

Selected items:
Syncreon1: 1.0
Syncreon2: 0.0
Ceva1: 